In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp indexers.faceclustering.models

In [ ]:
#export
from integrators.indexers.faceclustering.utils import *
from collections import Counter
from pathlib import Path
from mmcv.runner import load_checkpoint

import torch.nn as nn
import torch
import numpy as np
import scipy.sparse as sp
import cv2
import os
import torch.nn.functional as F

In [ ]:
# export
from torch.nn import init

class MeanAggregator(nn.Module):
    def __init__(self):
        super(MeanAggregator, self).__init__()

    def forward(self, features, A):
        if features.dim() == 2:
            x = torch.spmm(A, features)
        elif features.dim() == 3:
            x = torch.bmm(A, features)
        else:
            raise RuntimeError('the dimension of features should be 2 or 3')
        return x

class GraphConv(nn.Module):
    def __init__(self, in_dim, out_dim, agg, dropout=0):
        super(GraphConv, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.weight = nn.Parameter(torch.FloatTensor(in_dim * 2, out_dim))
        self.bias = nn.Parameter(torch.FloatTensor(out_dim))
        init.xavier_uniform_(self.weight)
        init.constant_(self.bias, 0)
        self.agg = agg()
        self.dropout = dropout

    def forward(self, features, A):
        feat_dim = features.shape[-1]
        assert (feat_dim == self.in_dim)
        agg_feats = self.agg(features, A)
        cat_feats = torch.cat([features, agg_feats], dim=-1)
        if features.dim() == 2:
            op = 'nd,df->nf'
        elif features.dim() == 3:
            op = 'bnd,df->bnf'
        else:
            raise RuntimeError('the dimension of features should be 2 or 3')
        out = torch.einsum(op, (cat_feats, self.weight))
        out = F.relu(out + self.bias)
        if self.dropout > 0:
            out = F.dropout(out, self.dropout, training=self.training)
        return out

In [ ]:
# export
class GCN_V(nn.Module):
    def __init__(self, feature_dim, nhid, nclass, dropout=0):
        super(GCN_V, self).__init__()
        self.conv1 = GraphConv(feature_dim, nhid, MeanAggregator, dropout)

        self.nclass = nclass
        self.classifier = nn.Sequential(nn.Linear(nhid, nhid), nn.PReLU(nhid),
                                        nn.Linear(nhid, self.nclass))
        self.loss = torch.nn.MSELoss()

    def forward(self, data, output_feat=False, return_loss=False):
        assert not output_feat or not return_loss
        x, adj = data[0], data[1]
        x = self.conv1(x, adj)
        pred = self.classifier(x).view(-1)

        if output_feat:
            return pred, x

        if return_loss:
            label = data[2]
            loss = self.loss(pred, label)
            return pred, loss

        return pred

In [ ]:
# export
def l2norm(vec):
    vec /= np.linalg.norm(vec, axis=1).reshape(-1, 1)
    return vec


class GCNVDataset(object):
    def __init__(self, features, label_path=None, k=80, feature_dim=256, is_norm_feat=True, th_sim=0.0, 
                 max_conn=1, conf_metric="s_nbr", eval_interim=True):
        
        self.inst_num, self.feature_dim = features.shape
        self.k = k
        self.is_norm_feat = is_norm_feat

        self.th_sim = th_sim
        self.max_conn = max_conn
        self.conf_metric = conf_metric
        self.features = l2norm(features)
        self.size = 1 # take the entire graph as input
            
        knns = build_knns("./data/knns/part1_test_small", self.features, "faiss", k, is_rebuild=True)

        adj = fast_knns2spmat(knns, k, th_sim, use_sim=True)

        # build symmetric adjacency matrix
        self.adj = row_normalize(build_symmetric_adj(adj, self_loop=True))

        # convert knns to (dists, nbrs)
        self.dists, self.nbrs = knns2ordered_nbrs(knns)

    def __getitem__(self, index):
        ''' return the entire graph for training.
        To accelerate training or cope with larger graph,
        we can sample sub-graphs in this function.
        '''

        assert index == 0
        return (self.features, self.adj_indices, self.adj_values,
                self.adj_shape, self.labels)

    def __len__(self):
        return self.size


# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()